In [ ]:
%load_ext autoreload
%autoreload 2

Please keep this notebook functionally at par with scripts/oncall/generate_report.py AND linearly runnable (no code with duplicated functionality; run all from this cell down should produce a report). Modify the parameters in `single_fitting_cycle` and `plot_trials` per customizations

In [ ]:
import numpy as np
import pandas as pd

import datetime
import copy
import json
import time
import os

import sys
sys.path.append('../../')

from data.dataloader import Covid19IndiaLoader
from data.processing import get_data
from data.processing import get_dataframes_cached

from models.seir import SEIR_Testing, SEIRHD, SEIR_Movement, SEIR_Movement_Testing

from main.seir.fitting import single_fitting_cycle, get_variable_param_ranges
from main.seir.sensitivity import calculate_sensitivity_and_plot
from main.seir.forecast import get_forecast, forecast_all_trials, trials_to_df, scale_up_testing_and_forecast
from main.seir.forecast import create_all_csvs, create_decile_csv, create_region_csv, write_csv
from main.seir.uncertainty import MCMCUncertainty
from utils.create_report import create_report
from utils.enums import Columns
from utils.loss import Loss_Calculator
from viz import plot_forecast, plot_top_k_trials, plot_ptiles

In [ ]:
t = time.time()
output_folder = f'../../reports/{t}'

## Params

In [ ]:
date_of_interest = '2020-07-31'
forecast_days = 54
now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
folder = str(now)
ktrials = 10
model=SEIRHD

In [ ]:
predictions_dict = {}

In [ ]:
mcmc_cfg = json.load(open('../../uncertainty/cfg/testing.json'))

## Select Districts to fit on

In [ ]:
state, district = ('Maharashtra', 'Mumbai')

## Perform M1 and M2 fits

In [ ]:
predictions_dict['m1'] = single_fitting_cycle(
    state, district, train_period=21, val_period=7, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=model, use_mcmc=True, mcmc_config=mcmc_cfg, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])
predictions_dict['m2'] = single_fitting_cycle(
    state, district, train_period=21, val_period=0, num_evals=1000,
    data_from_tracker=False, initialisation='intermediate', model=model, use_mcmc=True, mcmc_config=mcmc_cfg, 
    smooth_jump=True,
    which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered'])

predictions_dict['state'] = state
predictions_dict['dist'] = district
predictions_dict['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
predictions_dict['datasource'] = 'covid19api' if predictions_dict['m1']['data_from_tracker'] else 'municipality'
predictions_dict['variable_param_ranges'] = predictions_dict['m1']['variable_param_ranges']
predictions_dict['data_last_date'] = predictions_dict['m2']['data_last_date']

## Fitting Loss

### M1 Loss

In [ ]:
predictions_dict['m1']['df_loss']

### M2 Loss

In [ ]:
predictions_dict['m2']['df_loss']

## Sensitivity Plots

In [ ]:
# predictions_dict['m1']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m1')
# predictions_dict['m2']['plots']['sensitivity'], _, _ = calculate_sensitivity_and_plot(predictions_dict, which_fit='m2')

## Plot Forecasts

In [ ]:
predictions_dict['m2']['forecasts'] = {}
predictions_dict['m2']['forecasts']['best'] = get_forecast(predictions_dict, train_fit='m2', days=forecast_days)
predictions_dict['m2']['plots']['forecast_best'] = plot_forecast(predictions_dict, (state, district), error_bars=True)

trials_processed = forecast_all_trials(predictions_dict, train_fit='m1', forecast_days=forecast_days)
predictions_dict['m1']['trials_processed'] = trials_processed
predictions_dict['m1']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
trials_processed = forecast_all_trials(predictions_dict, train_fit='m2', forecast_days=forecast_days)
predictions_dict['m2']['trials_processed'] = trials_processed
predictions_dict['m2']['all_trials'] = trials_to_df(trials_processed, column=Columns.confirmed)
kforecasts = plot_top_k_trials(
    predictions_dict,
    train_fit='m2',
    trials_processed=predictions_dict['m2']['trials_processed'], 
    k=ktrials,
    which_compartments=[Columns.confirmed, Columns.active])
predictions_dict['m2']['plots']['forecast_confirmed_topk'] = kforecasts[Columns.confirmed]
predictions_dict['m2']['plots']['forecast_active_topk'] = kforecasts[Columns.active]

## Uncertainty

In [ ]:
uncertainty = MCMCUncertainty(predictions_dict, date_of_interest)

## Uncertainty Forecasts

In [ ]:
uncertainty_forecasts = uncertainty.get_forecasts()
for key in uncertainty_forecasts.keys():
    predictions_dict['m2']['forecasts'][key] = uncertainty_forecasts[key]['df_prediction']

In [ ]:
predictions_dict['m2']['deciles'] = uncertainty_forecasts

In [ ]:
predictions_dict['m2']['plots']['forecast_best_50'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 50], error_bars=False)
predictions_dict['m2']['plots']['forecast_best_80'] = plot_forecast(predictions_dict, (state, district), fits_to_plot=['best', 80], error_bars=False)

In [ ]:
plots = plot_ptiles(predictions_dict, which_compartments=[Columns.confirmed, Columns.active, Columns.daily_cases])
predictions_dict['m2']['plots']['forecast_confirmed_ptiles'] = plots[Columns.confirmed]
predictions_dict['m2']['plots']['forecast_active_ptiles'] = plots[Columns.active]
predictions_dict['m2']['plots']['forecast_new_cases_ptiles'] = plots[Columns.daily_cases]

## What-ifs

In [ ]:
scaling_factors = [1.2, 1.5, 1.8, 2.0]
scenario_on_which_df = 50
for factor in scaling_factors:
    ppfactor = str(factor).replace('.', '')
    predictions_dict['m2']['forecasts'][f'testing_{ppfactor}'] = scale_up_testing_and_forecast(predictions_dict, model=model, 
                                                                                               scenario_on_which_df=scenario_on_which_df, 
                                                                                               testing_scaling_factor=factor)

In [ ]:
predictions_dict['m2']['plots']['whatifs_testing'] = plot_forecast(predictions_dict, (state, district), 
                                                                   fits_to_plot=['testing_12', 'testing_15', 'testing_18',
                                                                                 'testing_20'], 
                                                                   error_bars=False)

## Create Report

In [ ]:
# create_report(predictions_dict, ROOT_DIR=output_folder)
# predictions_dict['m1']['all_trials'].to_csv(os.path.join(output_folder, 'm1-trials.csv'))
# predictions_dict['m2']['all_trials'].to_csv(os.path.join(output_folder, 'm2-trials.csv'))

## Create and Save Output CSV

In [ ]:
# df_final = create_all_csvs(predictions_dict)

In [ ]:
# write_csv(df_final, filename=os.path.join(output_folder, f'output-{t}.csv'))

In [ ]:
# df_output = create_decile_csv(predictions_dict, region='Mumbai', regionType='district')

In [ ]:
# write_csv(df_output, filename=os.path.join(output_folder, f'output-format-2-{t}.csv'))